#Analysis of Ad-Hoc Communications Network
The user can supply the overall size of desired coverage footprint andthen we will determine the following:

- Given an overall desired coverage footprint and a sequence of n communications
towers, what is the resulting resolved coverage?
- What is the total area of coverage relative to the desired total coverage area of the
original footprint? That is, are there any gaps in coverage?
- On average, how many communications towers are required before full coverage is
obtained?